# 환경 load & 라이브러리 임포트

In [1]:
import os
import langchain_chroma
import openai
import chromadb
import fitz  # PDF에서 텍스트 추출
from dotenv import load_dotenv
from fastapi import FastAPI, UploadFile, File
from sqlalchemy import create_engine, Column, Integer, String, Text, TIMESTAMP
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
from datetime import datetime
from sqlalchemy import create_engine, text
import pandas as pd
import psycopg2
from openai import OpenAI
# .env 파일 로드
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [2]:
POST_DB_HOST = os.getenv("POST_DB_HOST")
POST_DB_NAME = os.getenv("POST_DB_NAME")
POST_DB_USER = os.getenv("POST_DB_USER")
POST_DB_PASSWD = os.getenv("POST_DB_PASSWD")
POST_DB_PORT = os.getenv("POST_DB_PORT")

In [3]:
# PostgreSQL 연결 엔진 생성
post_engine = create_engine(f'postgresql://{POST_DB_USER}:{POST_DB_PASSWD}@{POST_DB_HOST}:{POST_DB_PORT}/{POST_DB_NAME}')
db = psycopg2.connect(host=POST_DB_HOST, dbname=POST_DB_NAME,user=POST_DB_USER,password=POST_DB_PASSWD,port=POST_DB_PORT)
cursor = db.cursor()
db.autocommit = False

---

## data 불러오기

In [4]:
sql = """
        select rec_idx, company_nm, jd_text from saramin_recruit_detail
"""

cursor.execute(sql)

In [5]:
jd_text_list = cursor.fetchall()

In [6]:
db.commit()
cursor.close()
db.close()

In [ ]:
# 텍스트 임베딩 함수
def get_embedding(text, model="text-embedding-3-large"):
    response = openai.embeddings.create(
        text = resume_text.replace("\n", "").strip(),
        model=model,
        input=text
    )
    return response.data[0].embedding

In [7]:
jd_text_list

[(50245530,
  '(주)하이로닉',
  '\n채용공고 상세\n[코스닥 상장사] 경영기획본부 신입2007년 12월 11일에 설립된 그 외 기타 의료용 기기 제조업업종의 피부미용 의료기기 제조,중개,무역사업을 하는 코스닥,중소기업,주식회사,외부감사법인,병역특례 인증업체,수출입 기업 입니다.모집부문 및 상세내용모집부문상세내용공통 자격요건ㆍ학력 : 대졸 이상 (4년)ㆍ학점 : 3.8 이상경영기획본부전략기획팀 1명주요업무담당업무ㆍ지원직무주요 회의체 운영 및 경영진 지원 (회의 안건 준비, 회의록 작성 등)조직별 성과 데이터 집계 및 분석, 경영진 보고사업 전략 및 중장기 계획 수립 지원기타 경영지원 및 프로젝트성 업무 수행지원자격데이터 분석 및 보고서 작성 능력 (Excel, PowerPoint 활용)커뮤니케이션 및 조율 역량 보유컨설팅 및 전략 기획 관련 경험 우대\nㆍ기타 필수 사항\n우대사항✅ 전략기획, 경영관리 등 유관 업무 경험 보유자✅ 데이터 분석 및 리포팅 역량 우수자 (BI 툴, SQL 활용 가능자 우대)✅ 사업 전략 및 중장기 계획 수립 경험자✅ 주요 프로젝트 운영 경험 보유자근무조건ㆍ근무형태:정규직(수습기간)-3개월ㆍ근무일시:주 5일(월~금)ㆍ근무지역:(16827) 경기 용인시 수지구 신수로 767 분당수지 U-TOWER 19층(동천동) - 신분당선 동천 에서 200m 이내전형절차 서류전형 1차면접 2차면접(경우에 따라 생략 가능) 최종합격접수기간 및 방법ㆍ:2025년 3월 14일 (금) 14시~ 채용시ㆍ접수방법:사람인 입사지원ㆍ이력서양식:사람인 온라인 이력서ㆍ제출서류:유의사항ㆍ학력, 성별, 연령을 보지않는 블라인드 채용입니다. ㆍ입사지원 서류에 허위사실이 발견될 경우, 채용확정 이후라도 채용이 취소될 수 있습니다.ㆍ모집분야별로 마감일이 상이할 수 있으니 유의하시길 바랍니다.\n'),
 (50275785, '세일즈웍스코리아(유)', '\n채용공고 상세\n'),
 (50275903,
  '엔에이치엔(주)',
  '\n채용공고 상세\nNHN\xa

In [8]:
jd_text_list[0][2]

'\n채용공고 상세\n[코스닥 상장사] 경영기획본부 신입2007년 12월 11일에 설립된 그 외 기타 의료용 기기 제조업업종의 피부미용 의료기기 제조,중개,무역사업을 하는 코스닥,중소기업,주식회사,외부감사법인,병역특례 인증업체,수출입 기업 입니다.모집부문 및 상세내용모집부문상세내용공통 자격요건ㆍ학력 : 대졸 이상 (4년)ㆍ학점 : 3.8 이상경영기획본부전략기획팀 1명주요업무담당업무ㆍ지원직무주요 회의체 운영 및 경영진 지원 (회의 안건 준비, 회의록 작성 등)조직별 성과 데이터 집계 및 분석, 경영진 보고사업 전략 및 중장기 계획 수립 지원기타 경영지원 및 프로젝트성 업무 수행지원자격데이터 분석 및 보고서 작성 능력 (Excel, PowerPoint 활용)커뮤니케이션 및 조율 역량 보유컨설팅 및 전략 기획 관련 경험 우대\nㆍ기타 필수 사항\n우대사항✅ 전략기획, 경영관리 등 유관 업무 경험 보유자✅ 데이터 분석 및 리포팅 역량 우수자 (BI 툴, SQL 활용 가능자 우대)✅ 사업 전략 및 중장기 계획 수립 경험자✅ 주요 프로젝트 운영 경험 보유자근무조건ㆍ근무형태:정규직(수습기간)-3개월ㆍ근무일시:주 5일(월~금)ㆍ근무지역:(16827) 경기 용인시 수지구 신수로 767 분당수지 U-TOWER 19층(동천동) - 신분당선 동천 에서 200m 이내전형절차 서류전형 1차면접 2차면접(경우에 따라 생략 가능) 최종합격접수기간 및 방법ㆍ:2025년 3월 14일 (금) 14시~ 채용시ㆍ접수방법:사람인 입사지원ㆍ이력서양식:사람인 온라인 이력서ㆍ제출서류:유의사항ㆍ학력, 성별, 연령을 보지않는 블라인드 채용입니다. ㆍ입사지원 서류에 허위사실이 발견될 경우, 채용확정 이후라도 채용이 취소될 수 있습니다.ㆍ모집분야별로 마감일이 상이할 수 있으니 유의하시길 바랍니다.\n'

In [9]:
df_rec = pd.DataFrame(jd_text_list, columns=['rec_id', 'company_name', 'description'])

In [10]:
df_rec

,rec_id,company_name,description
0,50245530,(주)하이로닉,\n채용공고 상세\n[코스닥 상장사] 경영기획본부 신입2007년 12월 11일에 설...
1,50275785,세일즈웍스코리아(유),\n채용공고 상세\n
2,50275903,엔에이치엔(주),\n채용공고 상세\nNHN Dooray!올인원 협업 도구 두레이와 전자결재/게시판...
3,50254512,(주)글로벌스탠다드테크놀로지,\n채용공고 상세\n모집부문 및 자격요건\n 모집부문\n 경력사항\n 담당업무\n자...
4,49874934,(주)에이치비투자그룹,\n채용공고 상세\n
...,...,...,...
13848,50185544,(주)핀다,\n채용공고 상세\n 구분\n 상세내용\nBusiness\nPO\n(비교대출)\n(...
13849,50170082,오픈헬스케어(주),\n채용공고 상세\n\t\n\t 오픈헬스케어(주) ㅣ 전략기획본부 투자팀 - 경력\...
13850,50090229,(주)두나미스자산운용,\n채용공고 상세\n \n (주)두나미스자산운용\n \n펀드마케팅 \n경력3년이상(...
13851,50109575,(주)에코앤드림,\n채용공고 상세\n재무기획(FP&A) 담당자 채용\n# 모집부문 \n모집부문\n담...


---

# chroma db 적재

In [11]:
from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [13]:
openai_client = OpenAI()
def get_embedding(text, model="text-embedding-3-small"):
    text = text.replace("\n", " ")
    return openai_client.embeddings.create(input=[text], model=model).data[0].embedding

In [ ]:
# 토큰나이징
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,      # 약 1000 토큰(약 300~400 단어)
    chunk_overlap=200,    # 일부 문맥 보존
    separators=["\n\n", "\n", ".", " ", ""]
)

def split_texts(row):
    chunks = text_splitter.split_text(row["description"])
    return [(row["rec_id"], row["company_name"], chunk) for chunk in chunks]

In [ ]:
chunk_data = []
for _, row in df.iterrows():
    chunk_data.extend(split_texts(row))

In [ ]:
def get_batch_embeddings(texts, model="text-embedding-3-small"):
    texts = [text.replace("\n", " ") for text in texts]
    response = client.embeddings.create(input=texts, model=model)
    return [r.embedding for r in response.data]

In [ ]:
chunk_df = pd.DataFrame(chunk_data, columns=["rec_id", "company_name", "chunk"])

In [ ]:
chunk_df['chunk']

In [ ]:
from tqdm import tqdm
batch_size = 100
all_embeddings = []

for i in tqdm(range(0, len(chunk_df), batch_size)):
    batch_texts = chunk_df["chunk"].iloc[i:i+batch_size].tolist()
    batch_embeddings = get_batch_embeddings(batch_texts)
    all_embeddings.extend(batch_embeddings)

chunk_df["embedding"] = all_embeddings

In [ ]:
chunk_df["embedding"] = chunk_df["chunk"].apply(get_embedding)

In [ ]:
chroma_client = chromadb.PersistentClient(path="./chroma_data")

In [ ]:
collection_name = "job_postings_openai"

In [ ]:
try:
    collection = chroma_client.get_collection(collection_name)
except:
    collection = chroma_client.create_collection(name=collection_name)

In [ ]:
# 오류발생 코드 : 토큰이 너무 길어서 RecursiveCharacterTextsplitter 사용하여 chunking
documents = df["description"].tolist()
metadatas = df[["rec_id", "company_name"]].to_dict(orient="records")
ids = df["rec_id"].astype(str).tolist()
embeddings = [get_embedding(text) for text in documents]

In [ ]:
df

---

# 다시 chromadb 유사도 검색 진행 코드

In [ ]:
df_grouped = df.groupby("rec_id").agg({
    "company_name": "first",
    "description": lambda x: "\n".join(x.dropna().astype(str))
}).reset_index()

In [ ]:
def is_meaningful(text):
    if not isinstance(text, str):
        return False
    text = text.strip()
    return len(text) > 30 and text != "채용공고 상세"

df = df[df["description"].apply(is_meaningful)]

In [ ]:
df

---

# Elastic Search

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk
es_client = Elasticsearch ("http://192.168.0.22:9200", basic_auth=("elastic", "ElastiC7276" ))
es_client.info()

In [ ]:
index_name = "job_description"

# 인덱스 생성 (없을 때만)
if not es_client.indices.exists(index=index_name):
    es_client.indices.create(
        index=index_name,
        body={
            "mappings": {
                "properties": {
                    "description": {"type": "text"},         # 토큰화 검색
                    "rec_id": {"type": "keyword"},           # 정렬/필터링
                    "company_name": {"type": "keyword"}
                }
            }
        }
    )

In [ ]:
# 데이터 bulk 업로드
actions = [
    {
        "_index": index_name,
        "_id": str(row["rec_id"]),
        "_source": {
            "rec_id": str(row["rec_id"]),
            "company_name": row["company_name"],
            "description": row["description"]
        }
    }
    for _, row in df.iterrows()
]

In [ ]:

bulk(es_client, actions)
print(f"✅ Elasticsearch에 {len(actions)}건 저장 완료.")